In [ ]:
pip install pandas

In [ ]:
pip install selenium

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support import expected_conditions as EC
import random
import itertools
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException, TimeoutException
from logging import raiseExceptions
from random import random

In [ ]:
service = Service()
options = webdriver.ChromeOptions()
options.add_extension("adblock.crx")
options.add_argument("--disable-application-cache")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-setuid-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.page_load_strategy = 'eager'
prefs = {
    "profile.managed_default_content_settings.images": 2,
    "profile.managed_default_content_settings.stylesheets": 2,
    "profile.managed_default_content_settings.plugins": 2,
    "profile.managed_default_content_settings.popups": 2,
    "profile.managed_default_content_settings.geolocation": 2,
    "profile.managed_default_content_settings.notifications": 2,
    "profile.managed_default_content_settings.media_stream": 2,
    "profile.managed_default_content_settings.javascript": 1,
    "profile.managed_default_content_settings.frame": 2
}

options.add_experimental_option("prefs", prefs)
options.set_capability("pageLoadStrategy", "eager")
options.add_experimental_option("prefs", prefs)

In [ ]:
driver = webdriver.Chrome(service=service, options=options)

In [ ]:
driver.get('https://www.justicaeleitoral.jus.br/fato-ou-boato/@@pesquisar?SearchableText=&Agencias%3Alist=Fato+ou+Fake&periodo_inicial=&periodo_final=#')
lista = ['#', '\'', '/', ',', '.', '', ']', '[', '(', ')', '*', '@', '!', '?', '|', ':', ';', '&', '–', '”', '“', '…', '=', ',', '.', '–', '"']
noticiasTreinamento = []
noticiasValidacao = []
aux = []
tamanhoNoticias = 0

for i in range(5):
    # Acha todos os cards da notícia, faz isso em cada pag
    try:
        newsCards = WebDriverWait(driver, 30).until(
            EC.presence_of_all_elements_located(
                (By.XPATH,
                    '//div[@class="check-labels"]//img[@src="https://www.justicaeleitoral.jus.br/fato-ou-boato/++theme++fatosboatos_v2/images/logos/cards/fato-fake.png"]'
                )
            )
        )
    except:
        raise Exception('elemento não encontrado')

    # Para cada card
    for card in newsCards:
        # Navega até o card e clica
        driver.execute_script("arguments[0].scrollIntoView({behavior:'smooth',block:'center'});", card)
        time.sleep(2)
        WebDriverWait(driver, 120).until(EC.element_to_be_clickable(card)).click()

        # Muda de janela
        WebDriverWait(driver, 120).until(EC.number_of_windows_to_be(2))
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(1)

        # Procura a notícia e armazena
        try:
            WebDriverWait(driver, 2).until(
                EC.presence_of_element_located((By.CLASS_NAME, "multicontent,amp-mode-mouse"))
            )

            time.sleep(2)
            newsElements = driver.find_elements(By.XPATH, "//p[@class=' content-text__container ']/em")

            for noticia in newsElements:
                aux = []
                if (random() < 0.8):
                    texto = noticia.text.strip() # adiciona a noticia
                    texto = texto.split()
                    for words in texto:
                        for char in lista:
                            words = words.replace(char, '')
                        aux.append(words.lower())

                    aux = [palavra for palavra in aux if palavra != '']
                    aux.insert(0, 'f')
                    noticiasTreinamento.append(aux)
                else:
                    texto = noticia.text.strip() # adiciona a noticia
                    texto = texto.split()
                    for words in texto:
                        for char in lista:
                            words = words.replace(char, '')
                        aux.append(words.lower())

                    aux = [palavra for palavra in aux if palavra != '']
                    aux.insert(0, 'f')
                    noticiasValidacao.append(aux)

            tamanhoNoticias += len(newsElements)

        except Exception as e:
            print("erro:", e)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            continue

        # Fecha a janela e procura a prox noticia
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        print(tamanhoNoticias)

    # Aperta o botao de proximo
    next_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//a[@class="next" and contains(@href, "b_start:int=")]'))
    )
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", next_button)
    next_button.click()

    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//div[@class="check-labels"]'))
    )

    # insert no index 0 f ou r
    # insert um random no 0

In [ ]:
arqOut = open('./TREINOnoticiasFalsasFatoFake.txt', 'w', encoding='utf-8')
for noticia in noticiasTreinamento:
    print(noticia, file=arqOut)
arqOut.close()

In [ ]:
arqOut = open('./VALIDACAOnoticiasFalsasFatoFake.txt', 'w', encoding='utf-8')
for noticia in noticiasValidacao:
    print(noticia, file=arqOut)
arqOut.close()